In [1]:
from rasterstats import zonal_stats
import geopandas as gpd
import pandas as pd

In [2]:
gdf_delta_formal = gpd.read_file('data/delta/IPTU-SP-diferenca-de-Área-Total-Construída-2017-ate-2020-por-distritos.gpkg')

In [3]:
rs_delta = zonal_stats('data/delta/IPTU-SP-diferenca-de-Área-Total-Construída-2017-ate-2020-por-distritos.gpkg', 
                 'data/delta/MDS-sem-veg-acrescimo-2017-2020-crivado-mascarado-preenchido.tiff',
                 stats=['sum', 'count', 'percentile_25', 'median', 'percentile_75', 'std', 'nodata'])

In [4]:
rs_2017 = zonal_stats('data/delta/IPTU-SP-diferenca-de-Área-Total-Construída-2017-ate-2020-por-distritos.gpkg', 
                'data/2017/BHM-2017-1m.tiff',
                 stats=['sum', 'count', 'percentile_25', 'median', 'percentile_75', 'std', 'nodata'])

In [5]:
df_delta_2017 = pd.concat([gdf_delta_formal, pd.DataFrame(rs_delta).add_suffix('_delta'), pd.DataFrame(rs_2017).add_suffix('_2017')], axis=1)

In [18]:
df_delta_2017.loc[:, 'total_m2_2017'] = df_delta_2017.sum_2017 / 3.40

In [19]:
df_delta_2017.columns

Index(['ds_codigo', 'ds_subpref', 'ds_cd_sub', 'ds_sigla', 'ds_nome', '2017',
       '2018', '2019', '2020', 'Área Total Construída', 'geometry',
       'count_delta', 'sum_delta', 'std_delta', 'median_delta',
       'percentile_25_delta', 'percentile_75_delta', 'nodata_delta',
       'count_2017', 'sum_2017', 'std_2017', 'median_2017',
       'percentile_25_2017', 'percentile_75_2017', 'nodata_2017',
       'acrescimo_lidar_m2', 'taxa_de_informalidade_2017_2020',
       'total_m2_2017'],
      dtype='object')

In [20]:
df_delta_2017.loc[:, 'acrescimo_lidar_m2'] = df_delta_2017.sum_delta / 3.40

In [21]:
df_delta_2017.loc[:, 'taxa_de_informalidade_2017_2020'] = df_delta_2017.acrescimo_lidar_m2 / df_delta_2017.loc[:, 'Área Total Construída']

In [23]:
df_delta_2017.loc[:, 'metros_informais_2017_2020'] = df_delta_2017.acrescimo_lidar_m2 - df_delta_2017.loc[:, 'Área Total Construída']

In [ ]:
df_delta_2017.loc[:, 'informalidade_geral_2020'] = (df_delta_2017.sum_2017 / 3.4) + df_delta_2017.acrescimo_lidar_m2 - df_delta_2017.loc[:, '2020']

In [22]:
df_delta_2017.to_file('data/delta/distritos_zonal_stats.gpkg', driver='GPKG')